In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from pprint import pprint

sys.path.append(str(Path("../").resolve()))
pprint(sys.path)

import df_file_interchange as fi

['/home/peterma/exio_dev/repo/df_file_interchange/scratch',
 '/home/peterma/miniconda3/envs/df-file-interchange/lib/python311.zip',
 '/home/peterma/miniconda3/envs/df-file-interchange/lib/python3.11',
 '/home/peterma/miniconda3/envs/df-file-interchange/lib/python3.11/lib-dynload',
 '',
 '/home/peterma/miniconda3/envs/df-file-interchange/lib/python3.11/site-packages',
 '/home/peterma/exio_dev/repo/df_file_interchange']


In [2]:
df = pd.DataFrame(np.random.randn(3, 4), columns=["a", "b", "c", "d"])
df["pop"] = pd.array([1234, 5678, 101010])
df

,a,b,c,d,pop
0,-1.515851,0.941612,-0.498784,2.118864,1234
1,-0.108423,0.148289,-0.637396,0.641605,5678
2,0.168237,-0.517011,0.102002,0.383239,101010


In [3]:
unit_cur_a = fi.ci.unit.currency.FICurrencyUnit(unit_desc="USD", unit_multiplier=1000)
unit_cur_b = fi.ci.unit.currency.FICurrencyUnit(unit_desc="EUR", unit_multiplier=1000)
unit_cur_c = fi.ci.unit.currency.FICurrencyUnit(
    unit_desc="JPY", unit_multiplier=1000000
)
unit_cur_d = fi.ci.unit.currency.FICurrencyUnit(unit_desc="USD", unit_multiplier=1000)
unit_pop = fi.ci.unit.population.FIPopulationUnit(
    unit_desc="people", unit_multiplier=1
)

extra_info = fi.ci.structured.FIStdExtraInfo(author="Spud", source="A simple test")

custom_info = fi.ci.structured.FIStructuredCustomInfo(
    extra_info=extra_info,
    col_units={
        "a": unit_cur_a,
        "b": unit_cur_b,
        "c": unit_cur_c,
        "d": unit_cur_d,
        "pop": unit_pop,
    },
)

In [4]:
issubclass(fi.ci.structured.FIStructuredCustomInfo, fi.file.rw.FIBaseCustomInfo)

True

In [5]:
metafile = fi.write_df_to_csv(df, Path("./test_with_structured_custom_info.csv"), custom_info=custom_info)

In [6]:
# %load_ext autoreload
# %autoreload 2

context = {
    "clss_custom_info": {
        "FIStructuredCustomInfo": fi.ci.structured.FIStructuredCustomInfo,
    },
    "clss_extra_info": {
        "FIStdExtraInfo": fi.ci.structured.FIStdExtraInfo,
    },
    "clss_col_units": {
        "FICurrencyUnit": fi.ci.unit.currency.FICurrencyUnit,
    },
}

(df_reload, metainfo_reload) = fi.read_df(metafile, context_metainfo=context)

In [7]:
metainfo_reload

FIMetainfo(datafile=PosixPath('test_with_structured_custom_info.csv'), file_format=<FIFileFormatEnum.csv: 'csv'>, format_version=1, hash='f5749ec2274d816fe01577267d582e61a05d5983cf4dc2d0e992da036634a0ab', encoding=FIEncoding(csv=FIEncodingCSV(csv_allowed_na=['<NA>'], sep=',', na_rep='<NA>', keep_default_na=False, doublequote=True, quoting=2, float_precision='round_trip'), parq=FIEncodingParquet(engine='pyarrow', index=None), auto_convert_int_to_intna=True), custom_info=FIStructuredCustomInfo(unstructured_data={}, extra_info=FIStdExtraInfo(author='Spud', source='A simple test', classname='FIStdExtraInfo'), col_units={'a': FICurrencyUnit(unit_desc='USD', unit_multiplier=1000.0, classname='FICurrencyUnit'), 'b': FICurrencyUnit(unit_desc='EUR', unit_multiplier=1000.0, classname='FICurrencyUnit'), 'c': FICurrencyUnit(unit_desc='JPY', unit_multiplier=1000000.0, classname='FICurrencyUnit'), 'd': FICurrencyUnit(unit_desc='USD', unit_multiplier=1000.0, classname='FICurrencyUnit')}, classname='F

In [8]:
metainfo_reload.custom_info

FIStructuredCustomInfo(unstructured_data={}, extra_info=FIStdExtraInfo(author='Spud', source='A simple test', classname='FIStdExtraInfo'), col_units={'a': FICurrencyUnit(unit_desc='USD', unit_multiplier=1000.0, classname='FICurrencyUnit'), 'b': FICurrencyUnit(unit_desc='EUR', unit_multiplier=1000.0, classname='FICurrencyUnit'), 'c': FICurrencyUnit(unit_desc='JPY', unit_multiplier=1000000.0, classname='FICurrencyUnit'), 'd': FICurrencyUnit(unit_desc='USD', unit_multiplier=1000.0, classname='FICurrencyUnit')}, classname='FIStructuredCustomInfo')